In [1]:
import numpy as np
import json
from styles import styles, styles_count
import copy
import itertools
import random


## Styles generator

In [4]:
class StyleGenerator:


    def __init__(self, styles, styles_count):
        self.styles = list(styles.items())

        data = copy.deepcopy(list(styles_count.values()))

        for i in range(len(data)):
            for j in range(len(data[i])):
                data[i][j] = data[i][j] + 1 #MLE

        for i in range(len(data)):
            for j in range(len(data[i])):
                data[i][j] = data[i][j]/sum(data[i])

        self.combs_indx = list(itertools.product(*[list(range(len(i))) for i in data]))

        self.weights = []
        for i in self.combs_indx:
            w = 1
            for j in range(len(i)):
                w *= data[j][i[j]]
            self.weights.append(w)
        
    def generate(self):
        idx = random.choices(list(range(len(self.combs_indx))), weights=self.weights, k=1)[0]
        comb = self.combs_indx[idx]
        res = ""
        for i in range(len(comb)):
            res = res + self.styles[i][0] + ': '
            res = res + self.styles[i][1][comb[i]] + '\n'
        return res, self.weights[idx]

        
        

In [5]:
generator = StyleGenerator(styles, styles_count)


In [6]:
for i in range(5):
    comb, prob = generator.generate()
    print(f"Style #{i}, prob={prob}")
    print(comb + '\n')

Style #0, prob=0.026645744545912533
прическа: нет волос
цвет волос: серебристо серый
аксесуар: солнцезащитные очки
одежда: комбинезон
цвет одежды: красный


Style #1, prob=0.035146291906090976
прическа: короткая прямые
цвет волос: рыжий
аксесуар: круглые очки
одежда: футболка с V-вырезом
цвет одежды: красный


Style #2, prob=0.1835853085249262
прическа: короткая прямые
цвет волос: серебристо серый
аксесуар: круглые очки
одежда: футболка с круглым вырезом
цвет одежды: белый


Style #3, prob=0.19331139668826777
прическа: короткая курчавые
цвет волос: серебристо серый
аксесуар: круглые очки
одежда: футболка с V-вырезом
цвет одежды: красный


Style #4, prob=0.08707812457143604
прическа: короткая курчавые
цвет волос: серебристо серый
аксесуар: круглые очки
одежда: футболка с V-вырезом
цвет одежды: зеленый




## Avatar generator

In [7]:
import os
import numpy as np
import cv2

In [16]:
class AvatarGenerator:
    """
    Генератор аватаров
    Значения каждого канала каждого пикселя генерируются отдельно
    """

    def __init__(self, images, mle=False):
        images = np.stack(images, axis=3)
        probs = np.zeros([*images.shape[:3], 256], dtype=np.float32)
        for i in range(images.shape[0]):
            for j in range(images.shape[1]):
                for k in range(images.shape[2]):
                    unique, unique_counts = np.unique(images[i, j, k], return_counts=True)
                    for t in range(unique.shape[0]):
                        probs[i, j, k, unique[t]] += unique_counts[t]
        #mle
        if mle:
            probs = probs + 1
        for i in range(images.shape[0]):
            for j in range(images.shape[1]):
                for k in range(images.shape[2]):
                    probs[i, j, k] /= np.sum(probs[i, j, k])
        self.probs = probs
        
    def generate(self):
        values = list(range(256))
        gen_img = np.zeros(self.probs.shape[:3], dtype=np.uint8)
        for i in range(self.probs.shape[0]):
            for j in range(self.probs.shape[1]):
                for k in range(self.probs.shape[2]):
                    val = random.choices(values, weights=self.probs[i, j, k].tolist(), k=1)[0]
                    gen_img[i, j, k] = val
                    
        return gen_img

        
        

Сначала проетстируем генерацию аватаров без MLE

In [17]:
image_names = os.listdir('./avatars')
images = []
for i in image_names:
    img = cv2.imread(os.path.join('./avatars', i))
    images.append(img)
generator = AvatarGenerator(images)

In [10]:
cv2.imwrite('test.png', generator.generate())
cv2.imwrite('test1.png', generator.generate())
cv2.imwrite('test2.png', generator.generate())

True

![title](test.png)

![title](test1.png)

![title](test2.png)

Теперь попробуем генерацию с MLE

Как можно заметить, для данного алгоритма генерации добавление MLE превращает сгенерированные аватары в шум

In [11]:
generator = AvatarGenerator(images, mle=True)
cv2.imwrite('test4_mle.png', generator.generate())
cv2.imwrite('test5_mle.png', generator.generate())

True

![title](test4_mle.png)

![title](test5_mle.png)